VERSION REQUIREMENTS:

* conda install numpy=1.22
* conda install -c conda-forge librosa
* pip install pydub
* pip install sounddevice
* pip install moviepy


In [ ]:
import os
import subprocess
from glob import glob
import librosa
import numpy as np
from pathlib import Path

from moviepy.editor import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio

import sounddevice as sd
from pydub import AudioSegment

from scipy.signal import correlate, resample
from tqdm import tqdm
from IPython.display import Audio, Video
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


In [ ]:
'''
The following functions were written by Landry Bulls.
See full code at https://github.com/LandryBulls/multidata/blob/main/alignment/align.py
'''

def extract_audio(posix_video_path):
    out_audio_path = str(os.path.basename(str(posix_video_path)[:-4]+'.wav'))
    ffmpeg_extract_audio(str(posix_video_path), out_audio_path)
    # librosa returns the numpy array as well as the sample rate
    loaded = librosa.load(out_audio_path, sr=None)
    # delete the file
    os.remove(out_audio_path)
    # returns both the audio as a numpy array and the sample rate (arr, sr)
    return loaded

def mix_audio(list_of_arrays):
    """
    Takes a list of numpy arrays and mixes them together, padding the shorter arrays with zeros.
    """
    # find the longest array
    max_len = max([len(arr) for arr in list_of_arrays])
    # pad the shorter arrays with zeros
    padded_arrays = [np.pad(arr, (0, max_len-len(arr)), 'constant') for arr in list_of_arrays]
    # sum the arrays
    return np.sum(padded_arrays, axis=0)

def get_shift_values(list_of_camera_audio_arrays, microphone_mix):
    """
    Takes a list of numpy arrays representing audio from each camera and a numpy array representing the microphone mix.
    Returns a list of numpy arrays representing the aligned audio from each camera.
    """
    # cross-correlate each audio stream with the mix
    # the output array represents how similar the two signals are at each time step
    all_arrays = list_of_camera_audio_arrays + [microphone_mix]

    # find the longest array
    max_pos = np.argmax(np.array([len(arr) for arr in all_arrays]))
    maxlen = len(all_arrays[max_pos])
    del all_arrays

    # pad the shorter arrays with zeros    
    list_of_camera_audio_arrays = [np.pad(arr, (0, maxlen-len(arr)), 'constant') for arr in list_of_camera_audio_arrays]
    microphone_mix = np.pad(microphone_mix, (0, maxlen-len(microphone_mix)), 'constant')

    # cross-correlate each array with the microphone mix
    print("Cross-correlating audio streams with microphone mix...\n")
    correlations = [correlate(arr, microphone_mix, mode='full') for arr in list_of_camera_audio_arrays]

    # find the time shift (in seconds) that maximizes the correlation
    print("Finding time shift that maximizes correlation...\n")
    shifts = [np.argmax(corr) - len(arr) for corr, arr in zip(correlations, list_of_camera_audio_arrays)]
    for s, shift in enumerate(shifts):
        print(f'Shift {s}: {shift} samples')
    return shifts

def trim_video(posix_video_path, start_time, end_time, output_path):
    """
    Takes a posix path to a video file, a start trim value (in seconds), an end trim value (in seconds), and an output path.
    Trims the video and saves it to the output path.
    """
    stringPath = str(posix_video_path)
    duration = VideoFileClip(stringPath).duration
    ffmpeg_command = f'ffmpeg -i {stringPath} -ss {start_time} -to {end_time} -c copy {output_path}' ###
    subprocess.run(ffmpeg_command, shell=True)

def trim_audio(posix_audio_path, start_time, end_time, output_path):
    """
    Takes a posix path to an audio file, a start trim value (in seconds), an end trim value (in seconds), and an output path.
    Trims the audio and saves it to the output path.
    """
    stringPath = str(posix_audio_path)
    duration = librosa.get_duration(filename=stringPath)
    ffmpeg_command = f'ffmpeg -i {stringPath} -ss {start_time} -to {end_time} -c copy {output_path}'
    subprocess.run(ffmpeg_command, shell=True)

In [ ]:
path = ''
data_dir = os.listdir(path)
pairlist = [x for x in data_dir if 'pair-' in x]
pairlist.sort() #pair-1

# <font color='deeppink'>Aligning audio and brio files.</font>

In [ ]:
for pair in pairlist:
    print("starting pair: ", pair)

    try:
        pair_path = path+pair+os.sep
        derivative_path = pair_path+"derivatives"
        
        audio_list = os.listdir(pair_path+"audio/")
        audio_paths = [pair_path+"audio/"+x for x in audio_list if ".WAV" in x]
        
        #gopro_file = pair_path+"gopro/gopro_concatenated.mp4"
        
        brio_list = os.listdir(pair_path+"brio/")
        video_paths = [pair_path+"brio/"+x for x in brio_list]
        
        #video_paths.append(gopro_file)
        
        # make mix of all mics
        print("Starting the mixing process...")
        mic_mix = mix_audio([librosa.load(mic, sr=None)[0] for mic in audio_paths])
        mic_audio_sr = librosa.load(audio_paths[0], sr=None)[1]
        print("Mix created.")
        
        # extract audio from video files
        print("Extracting audio from video files...\n")
        video_audio = [extract_audio(vid) for vid in video_paths]
        video_audio_sr = video_audio[0][1]
        video_audio = [i[0] for i in video_audio]   
        print("Audio extracted!")

        assert mic_audio_sr == video_audio_sr, "Sample rates of audio files do not match."

        # get durations of video files
        print("Getting durations of video files...\n")
        video_durations = dict(zip(video_paths, [VideoFileClip(str(vid)).duration for vid in video_paths]))
        audio_duration = mic_mix.shape[0]/mic_audio_sr

        # get shift values
        print("Getting shift values...\n")
        shift_values = get_shift_values(video_audio, mic_mix)

        # check distribution of positive and negative shifts
        if all([shift > 0 for shift in shift_values]):
            trim_status = 'early'
        elif all([shift < 0 for shift in shift_values]):
            trim_status = 'late'
        else:
            trim_status = 'mixed'

        start_end_keys = dict(zip(['in', 'out'], [[], []]))
        all_files = video_paths + ['audio_trim']
        trim_times = {}
        for file in all_files:
            trim_times[file] = start_end_keys.copy()
        trim_times['audio_trim'] = start_end_keys.copy()
            
        if trim_status == 'early':
            # trim the beginning of the video files
            for vid, shift in zip(video_paths, shift_values):
                trim_amount = shift/video_audio_sr
                trim_times[vid]['in'] = trim_amount
                video_durations[vid] = video_durations[vid] - trim_amount
                
            trim_times['audio_trim']['in'] = 0
        
        elif trim_status == 'late':
            # trim the end of the video files
            latest = min(shift_values)
            
            for vid, shift in zip(video_paths, shift_values):
                if shift == latest:
                    trim_times[vid]['in'] = 0
                else:
                    trim_amount = (abs(latest) - abs(shift))/video_audio_sr
                    trim_times[vid]['in'] = trim_amount
                    video_durations[vid] = video_durations[vid] - trim_amount
            
            audio_trim = abs(latest)/mic_audio_sr
            trim_times['audio_trim']['in'] = audio_trim
            audio_duration = audio_duration - audio_trim
        
        elif trim_status == 'mixed':
            latest = min(shift_values)
            for vid, shift in zip(video_paths, shift_values):
                if shift == latest:
                    trim_times[vid]['in'] = 0
                elif shift > 0:
                    trim_amount = (abs(shift)+abs(latest))/video_audio_sr
                    trim_times[vid]['in'] = trim_amount
                    video_durations[vid] = video_durations[vid] - trim_amount
                else:
                    trim_amount = (abs(latest) - abs(shift))/video_audio_sr
                    trim_times[vid]['in'] = trim_amount
                    video_durations[vid] = video_durations[vid] - trim_amount
            
            audio_trim = abs(latest)/mic_audio_sr
            trim_times['audio_trim']['in'] = audio_trim
            audio_duration = audio_duration - audio_trim
        
        # get ending trim times
        all_durations = list(video_durations.values()) + [audio_duration]    
        shortest = min(all_durations)
        for file in trim_times.keys():
            trim_times[file]['out'] = shortest + trim_times[file]['in']
            
        # trim the video files
        print("Trimming video files...\n")
        # all the trim times are in seconds
        for vid in video_paths:
            trim_video(vid, trim_times[vid]['in'], trim_times[vid]['out'], str(derivative_path+os.sep+f'{os.path.basename(str(vid))[:-4]}_trimmed.mp4'))
            
        # trim the audio files
        print("Trimming audio files...\n")
        for mic in audio_paths:
            trim_audio(mic, trim_times['audio_trim']['in'], trim_times['audio_trim']['out'], str(derivative_path+os.sep+f'{os.path.basename(str(mic))[:-4]}_trimmed.wav'))

        # print out all the trim times per file
        print("Trim times:\n")
        for file in trim_times.keys():
            print(f"{file}: {trim_times[file]['in']}s to {trim_times[file]['out']}s")

        print("DONE!")
    
    except:
        print("SOMETHING is amiss")
        continue
        